# 04 · Time Series Forecasting
Basic forecasting of turbidity and nitrate using statsmodels ARIMA (per station).

In [ ]:

import pandas as pd
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

DATA_DIR = Path("../data")
df = pd.read_csv(DATA_DIR / "water_quality_samples.csv", parse_dates=["timestamp"])

# Downsample to daily means per station
daily = df.set_index("timestamp").groupby("station_id").resample("D").mean(numeric_only=True).reset_index()

from statsmodels.tsa.arima.model import ARIMA

forecasts = []

for st in daily["station_id"].unique():
    for param in ["turbidity_NTU","nitrate_mgL"]:
        d = daily[daily["station_id"]==st].set_index("timestamp")[param].dropna()
        if len(d) < 20:
            continue
        model = ARIMA(d, order=(1,1,1))
        fit = model.fit()
        fcast = fit.forecast(steps=7)
        for ts, val in fcast.items():
            forecasts.append({"station_id": st, "parameter": param, "date": ts.date().isoformat(), "forecast": float(val)})

pd.DataFrame(forecasts).head(14)
